<a href="https://colab.research.google.com/github/aravind598/Data_Science_Proj/blob/master/Data%20Science%20Project/Deep_Learning_EarthQuake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://github.com/Goodsea/Richter-s-Eye/blob/master/main.ipynb

In [13]:
!pip install tensorflow==1.1
import numpy as np
import pandas as pd

import lightgbm as lgb

import keras 
from keras.layers import *
from keras.optimizers import *
from keras.models import Model

from sklearn.metrics import f1_score 
from sklearn.model_selection import KFold
!pip install tensorflow==1.1

In [0]:
url = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/train_labels.csv?token=ANW7SFBKZIJTONPGC67MZMS6SCJR2'
urll = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/train_values.csv?token=ANW7SFBK2YMNTYBWW44PE7C6SCJSE'
url2 = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/test_values.csv?token=ANW7SFHJZIY2XRWW5QVSW4S6SCS5I'

In [0]:
train_x = pd.read_csv(urll)
train_y = pd.read_csv(url)
test_x  = pd.read_csv(url2)

In [0]:
geo1 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_1_id"], test_x["geo_level_1_id"]])))
geo2 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_2_id"], test_x["geo_level_2_id"]])))
geo3 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_3_id"], test_x["geo_level_3_id"]])))

In [7]:
geo3

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [17]:
def NET():
    inp = Input((geo3.shape[1],))
    i1 = Dense(21, name="intermediate")(inp)
    x2 = Dense(geo2.shape[1], activation='sigmoid')(i1)
    x1 = Dense(geo1.shape[1], activation='sigmoid')(i1)

    model = Model(inp, [x2,x1])
    model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
    return model

model = NET()
model.fit(geo3, [geo2, geo1], batch_size=128, epochs=15, verbose=1)

Epoch 1/15
347469/347469 [==============================] - 95s 275us/step - loss: 0.1919 - dense_7_loss: 0.0482 - dense_8_loss: 0.1437 - dense_7_acc: 0.9989 - dense_8_acc: 0.9671
Epoch 2/15
347469/347469 [==============================] - 95s 274us/step - loss: 0.0396 - dense_7_loss: 0.0050 - dense_8_loss: 0.0346 - dense_7_acc: 0.9993 - dense_8_acc: 0.9907
Epoch 3/15
347469/347469 [==============================] - 96s 275us/step - loss: 0.0105 - dense_7_loss: 0.0040 - dense_8_loss: 0.0066 - dense_7_acc: 0.9993 - dense_8_acc: 0.9985
Epoch 4/15
347469/347469 [==============================] - 95s 274us/step - loss: 0.0053 - dense_7_loss: 0.0033 - dense_8_loss: 0.0019 - dense_7_acc: 0.9993 - dense_8_acc: 0.9996
Epoch 5/15
347469/347469 [==============================] - 94s 271us/step - loss: 0.0035 - dense_7_loss: 0.0027 - dense_8_loss: 7.6043e-04 - dense_7_acc: 0.9994 - dense_8_acc: 0.9998
Epoch 6/15
347469/347469 [==============================] - 95s 275us/step - loss: 0.0023 - dens

In [0]:
# "Extract Intermediate Layer" Function
from keras import backend as K

get_int_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

In [0]:
# Extract GEO-Embeds for all train data points.
# Then assign with train_data

out = []
for dat in geo3[:260601]:
    layer_output = get_int_layer_output([[dat]])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

train_data = pd.get_dummies(train_x.copy())
train_data = train_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
train_data = train_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15],
                               geo_feat17=out[:,16],
                               geo_feat18=out[:,17],
                               geo_feat19=out[:,18],
                               geo_feat20=out[:,19],
                               geo_feat21=out[:,20])

In [20]:
train_data.head()

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,...,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w,geo_feat1,geo_feat2,geo_feat3,geo_feat4,geo_feat5,geo_feat6,geo_feat7,geo_feat8,geo_feat9,geo_feat10,geo_feat11,geo_feat12,geo_feat13,geo_feat14,geo_feat15,geo_feat16,geo_feat17,geo_feat18,geo_feat19,geo_feat20,geo_feat21
0,802906,2,30,6,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.135093,-1.534365,-1.348366,0.786060,1.999220,-0.003626,0.176927,0.561674,1.122262,-0.749189,-1.310260,1.429933,0.820919,2.220829,-1.318938,-0.699338,0.313279,-1.974195,-1.281655,1.245836,-2.525863
1,28830,2,10,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1.453936,-0.971339,-2.045273,-0.770744,-0.197056,-2.206620,-1.508639,0.577139,0.706168,-0.310007,-0.784029,0.246667,-1.121311,1.265126,-0.240064,0.733916,-0.725880,-1.581892,-1.237335,1.660036,-0.519317
2,94947,2,10,5,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1.770885,-1.937941,-2.176445,0.226858,1.473619,-2.084702,-2.153265,-0.133046,-0.787757,-1.792517,-1.512898,-0.919633,-1.278114,2.020105,-0.580711,-1.694477,-1.066352,-2.263102,0.661076,1.622733,-1.858136
3,590882,2,10,6,5,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-0.231253,-1.383288,-1.763033,1.072076,1.402425,-2.314131,-1.706700,1.718395,-1.113076,-0.983362,-1.314574,1.797462,-0.165014,1.925920,-1.170168,0.777712,-1.204853,0.767948,-1.437132,0.156538,-2.680001
4,201944,3,30,8,9,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.582603,-1.567887,-1.994085,0.993428,2.135461,-2.048608,-2.141873,0.170889,0.815207,-1.584817,0.734021,2.063339,-0.836162,2.358575,0.195004,-1.691247,-0.606138,-0.810207,-1.212808,-0.075289,-2.568868


In [0]:
# Extract GEO-Embeds for all test data points.
# Then assign with train_data

out = []
for dat in geo3[260601:]:
    layer_output = get_int_layer_output([[dat]])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

test_data = pd.get_dummies(test_x.copy())
test_data = test_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
test_data = test_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15],
                               geo_feat17=out[:,16],
                               geo_feat18=out[:,17],
                               geo_feat19=out[:,18],
                               geo_feat20=out[:,19],
                               geo_feat21=out[:,20])

In [35]:
from google.colab import drive
drive.mount('drive')
test_data.to_csv('test_data.csv')
!cp test_data.csv "drive/My Drive/Colab Notebooks"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [36]:
from google.colab import drive
drive.mount('drive')
train_data.to_csv('train_data.csv')
!cp train_data.csv "drive/My Drive/Colab Notebooks"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
def threshold_arr(array):
    # Get major confidence-scored predicted value.
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val))))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr)

In [28]:
y = np.array(train_y["damage_grade"])-1

df = train_data.drop(["building_id"], axis=1)
x = np.array(df)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for ix, (train_index, test_index) in enumerate(kf.split(x)):
    lgb_params = {
        "objective" : "multiclass",
        "num_class":3,
        "metric" : "multi_error",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "feature_fraction" : 0.5,
        "min_sum_hessian_in_leaf" : 0.1,
        "max_bin":8192,
        "verbosity" : 1,
        "num_threads":6,
      
    }

    x_train, x_val, y_train, y_val= x[train_index], x[test_index], y[train_index], y[test_index]

    train_data = lgb.Dataset(x_train, label=y_train)
    val_data   = lgb.Dataset(x_val, label=y_val)

    lgb_clf = lgb.train(lgb_params,
                        train_data,
                        20000,
                        valid_sets = [val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000)

    y_pred = lgb_clf.predict(x_val)
    print("F1-MICRO SCORE: ", f1_score(np.array(pd.get_dummies(y_val)), threshold_arr(y_pred), average='micro'))
    

Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's multi_error: 0.25107
[2000]	valid_0's multi_error: 0.249688
[3000]	valid_0's multi_error: 0.250456
[4000]	valid_0's multi_error: 0.251089
[5000]	valid_0's multi_error: 0.253238
Early stopping, best iteration is:
[2202]	valid_0's multi_error: 0.248844
F1-MICRO SCORE:  0.7511559640068302
Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's multi_error: 0.250058
[2000]	valid_0's multi_error: 0.247544
[3000]	valid_0's multi_error: 0.248599
[4000]	valid_0's multi_error: 0.249271
Early stopping, best iteration is:
[1932]	valid_0's multi_error: 0.246662
F1-MICRO SCORE:  0.7533384497313892
Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's multi_error: 0.250537
[2000]	valid_0's multi_error: 0.248715
[3000]	valid_0's multi_error: 0.248868
[4000]	valid_0's multi_error: 0.250767
Early stopping, best iteration is:
[1508]	valid_0's multi_error: 0.247851
F1-MICRO